# Matching Rules Engine #
To be implementable as a serverless/Lambda function

Expected inputs via json:
 - Rules object relating tags to one another (e.g., pair of tags with relationship like "excluding")
 - People objects with a list of tags for each (IDs rather than strings)

First pass will be to adapt Wes's table-driven algorithm (where a subject like "dog" has two available columns, "dog_has" and "dog_conflict") to the tags model.
I.e., there is no "dog" root with two related columns. There can be as many tags related to dogs as you want, with separate rules relating all tags.

### Sample People

In [1]:
people_as_dicts = [
    {
        'tags': ['offender','dog-has','smoker'],
        'id': 'X001'
    },
    {
        'tags': ['offender','dog-has','smoker'],
        'id': 'X002'
    },
    {
        'tags': ['offender','dog-has','smoker'],
        'id': 'X003'
    },
    {
        'tags': ['dog-has','smoker'],
        'id': 'X004'
    },
    {
        'tags': ['dog-has','smoker'],
        'id': 'X005'
    },
    {
        'tags': ['dog-has','smoker'],
        'id': 'X006'
    },
    {
        'tags': ['dog-has','smoker'],
        'id': 'X007'
    },
    {
        'tags': ['dog-has','smoker'],
        'id': 'X008'
    },
    {
        'tags': ['dog-has','smoker'],
        'id': 'X009'
    },
    {
        'tags': ['dog-conflict','smoker'],
        'id': 'X010'
    },
    {
        'tags': ['dog-conflict','smoker'],
        'id': 'X011'
    },
    {
        'tags': ['dog-conflict','smoker'],
        'id': 'X012'
    },
    {
        'tags': ['dog-has','smoking-conflict'],
        'id': 'X013'
    },
    {
        'tags': ['dog-has','smoking-conflict'],
        'id': 'X014'
    },
    {
        'tags': ['dog-has','smoking-conflict'],
        'id': 'X015'
    },
    {
        'tags': ['dog-has','smoking-conflict'],
        'id': 'X016'
    },
    {
        'tags': ['dog-has','smoking-conflict'],
        'id': 'X017'
    },
    {
        'tags': ['dog-has'],
        'id': 'X018'
    },
    {
        'tags': ['dog-has'],
        'id': 'X019'
    },
    {
        'tags': ['dog-has'],
        'id': 'X020'
    }
]

### Sample Rules Data
This list of tuples will later be loaded from a json input

In [2]:
rules_by_label = [('dog-has', 'dog-conflict'),('smoker','smoking-conflict')]

### Build list of relevant tags
The point here is to only store data relevant to rules. We don't need a master list of all tags in the system (which will grow over time). So let's gather the tags that we have rules about.

In [3]:
relevant_tags = []
for rule in rules_by_label:
    for i in range(len(rule)):
        if rule[i] not in relevant_tags:
            relevant_tags.append(rule[i])
relevant_tags


['dog-has', 'dog-conflict', 'smoker', 'smoking-conflict']

### Build people_as_lists where each person is a list of 0s and 1s corresponding to that index in the relevant_tags array

In [4]:
people_as_lists = []

for person in people_as_dicts:
    peep_tags = []
    for i in range(len(relevant_tags)):
        if relevant_tags[i] in person['tags']:
            peep_tags.append(1)
        else:
            peep_tags.append(0)
    people_as_lists.append(peep_tags)

people_as_lists

[[1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [1, 0, 1, 0],
 [0, 1, 1, 0],
 [0, 1, 1, 0],
 [0, 1, 1, 0],
 [1, 0, 0, 1],
 [1, 0, 0, 1],
 [1, 0, 0, 1],
 [1, 0, 0, 1],
 [1, 0, 0, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 0]]

### Build rules by index-reference to relevant_tags


In [5]:
# loop through rules_by_label
rules_by_index = [(relevant_tags.index(rule[0]), relevant_tags.index(rule[1])) for rule in rules_by_label]

rules_by_index

[(0, 1), (2, 3)]

### Time to hard-group some folks!
Without their IDs, how will I relate the person_as_list back to the people_as_dicts?
 - The index is the same in both list and dicts versions, so can get back to IDs that way when reporting groupings

Note on output: groups will be lists of lists, with people known by their index in people_as_lists

In [31]:
groups = []
for peep_index, peep in enumerate(people_as_lists):  # loop thru people
#     max_group = len(groups)
    found_group = False
    if len(groups) == 0:  # first time through, start Group 0
        groups.append([peep_index])
        print('peep 0 found group 0', peep)
        print("groups",groups)
    else:
        # loop thru groups, look for conflicts
        for check_group in range(len(groups)):  # check_group is index of [group]
            group = groups[check_group]
            group_conflict = False
            # loop through members of group
            for j in range(len(group)):
                groupie = people_as_lists[group[j]]  # groupie = "person number j" in group
                # loop through rules in rules_by_index, check compatibility
                for rule in rules_by_index:  # a tuple with two ints
                    if ( (peep[rule[0]]*groupie[rule[1]]) + (peep[rule[1]]*groupie[rule[0]]) ):
                        group_conflict = True
                        print('group conflict, rule:',rule)
                        break   # don't bother checking more rules
                if group_conflict: 
                    break   # don't bother checking other group members, go to next group
            if not group_conflict:  # peep found their group, don't check more groups
                # add peep's index i to groups[check_group]
                groups[check_group].append(peep_index)
                found_group = True
                break   
        # if all groups failed...
        print(peep_index, 'found_group', peep)
        if not found_group:
            #print(f'new group please for {people_as_dicts[peep_index]['id']}:', max_group + 1)
            groups.append([peep_index])
            print("groups",groups)
groups
       
     

peep 0 found group 0 [1, 0, 1, 0]
groups [[0]]
1 found_group [1, 0, 1, 0]
2 found_group [1, 0, 1, 0]
3 found_group [1, 0, 1, 0]
4 found_group [1, 0, 1, 0]
5 found_group [1, 0, 1, 0]
6 found_group [1, 0, 1, 0]
7 found_group [1, 0, 1, 0]
8 found_group [1, 0, 1, 0]
group conflict, rule: (0, 1)
9 found_group [0, 1, 1, 0]
groups [[0, 1, 2, 3, 4, 5, 6, 7, 8], [9]]
group conflict, rule: (0, 1)
10 found_group [0, 1, 1, 0]
group conflict, rule: (0, 1)
11 found_group [0, 1, 1, 0]
group conflict, rule: (2, 3)
group conflict, rule: (0, 1)
12 found_group [1, 0, 0, 1]
groups [[0, 1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11], [12]]
group conflict, rule: (2, 3)
group conflict, rule: (0, 1)
13 found_group [1, 0, 0, 1]
group conflict, rule: (2, 3)
group conflict, rule: (0, 1)
14 found_group [1, 0, 0, 1]
group conflict, rule: (2, 3)
group conflict, rule: (0, 1)
15 found_group [1, 0, 0, 1]
group conflict, rule: (2, 3)
group conflict, rule: (0, 1)
16 found_group [1, 0, 0, 1]
17 found_group [1, 0, 0, 0]
18 found_gr

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 17, 18, 19], [9, 10, 11], [12, 13, 14, 15, 16]]

### Build groups_by_id to report groupings back out


In [44]:
groups_by_id = {'group' + str(j): [people_as_dicts[group[i]]['id'] for i in range(len(groups[j]))] for j in range(len(groups))}
groups_by_id

{'group0': ['X001',
  'X002',
  'X003',
  'X004',
  'X005',
  'X006',
  'X007',
  'X008',
  'X009',
  'X018',
  'X019',
  'X020'],
 'group1': ['X001', 'X002', 'X003'],
 'group2': ['X001', 'X002', 'X003', 'X004', 'X005']}